# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 25

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(15313, 264)

In [4]:
# calculate player form
for i in [5,10,15]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )
    # dirty fix in case there are inf results
    all_df.loc[(all_df['gameweek']==gameweek) & np.isinf(all_df[column_name]), column_name] = np.nan

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10']].sort_values(by=['form 5'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10
id,,,,,,,,
302,Fernandes,25.000000,190,7.6,170.567043,6.822682,7.794167,6.733575
112,Arrizabalaga,3.947368,15,3.8,17.292406,4.380743,6.986341,6.986341
239,Justin,22.954545,101,4.4,91.574010,3.989363,6.618239,5.232324
271,Gündogan,19.090909,126,6.6,109.842652,5.753663,6.578017,6.510632
120,Mount,24.146341,99,4.1,103.074938,4.268760,6.076950,4.909876
67,Veltman,18.750000,60,3.2,67.729127,3.612220,6.041449,4.770726
466,Neves,23.870968,74,3.1,77.842142,3.260955,6.010927,4.970890
102,Azpilicueta,15.000000,69,4.6,58.797575,3.919838,6.007802,4.891239
12,Martínez,22.807018,130,5.7,107.462550,4.711819,5.986852,4.658143


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)